# Import Libraries

In [34]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import random
#import cv2
import skimage.morphology as morp
from skimage.filters import rank
from sklearn.utils import shuffle
import csv
import os
#import pytorch
#import tensorflow as tf
from tensorflow.contrib.layers import flatten
from sklearn.metrics import confusion_matrix

# Load Data

In [35]:
# obtain data path
images_mali = np.load('images_mali.npy')
images_benign = np.load('images_benign.npy')
#validation_file= "./NAME
#testing_file = "./NAME"
print(images_mali.shape)
print(images_benign.shape)

image_original = np.vstack((images_mali,images_benign))
#image_sample = random.sample(image_original, 600)
#print(type(image_original))
#train = image_sample[:400]
#test = image_sample[401:]
## mali: 0
## benign: 1
x_label = np.zeros((196,1))
y_label = np.ones((421,1))
print(train.shape)
labels = np.vstack((x_label,y_label))
print(labels.shape)
print(train.shape)
#print(test.shape)
#print(images_mali[1])

(196, 7500)
(421, 7500)
(617, 7500)
(617, 1)
(617, 7500)


In [36]:
# total number: 617
# number of training samples: 400
# size of x_train = 200
# size of y_train = 200
# number of testing samples: 217
# size of x_test = 108
# size of y_test = 109

train_percentage = 0.8
test_percentage = 0.2

# split into x and y for featurs and labels
xtrain_idx, ytrain_idx = random.sample(range(int(196*train_percentage)), 150), \
random.sample(range(int(196*train_percentage),int(196*train_percentage)+421), 150)

xtest_idx, ytest_idx = random.sample(range(int(196*train_percentage),196), 30), \
random.sample(range(int(196*train_percentage)+421, 617), 30)

x_train = image_original[xtrain_idx,:]
y_train = image_original[ytrain_idx,:]
x_test = image_original[xtest_idx,:]
y_test = image_original[ytest_idx,:]
x_label = labels[x_idx,:]
y_label = labels[y_idx,:]

print("Number of x_training examples: ", x_train.shape)
print("Number of y_training examples: ", y_train.shape)

print("Number of x_testing examples: ", x_test.shape)
print("Number of y_testing examples: ", y_test.shape)
#print("Number of testing examples: ", n_test)
#print("Number of validation examples: ", n_validation)
#print("Image data shape =", image_shape)
print("Number of classes =", 2)

Number of x_training examples:  (150, 7500)
Number of y_training examples:  (150, 7500)
Number of x_testing examples:  (30, 7500)
Number of y_testing examples:  (30, 7500)
Number of classes = 2


# Display images

In [40]:
# function to display the images
def list_images(dataset, dataset_y, ylabel="", cmap=None):
    
    ## change the figure size to reduce features.
    #plt.figure(figsize=(15, 16))
    plt.figure(figsize = (50, 50))
    
    # n images to show.
    for i in range(n):
        plt.subplot(1, n, i+1)
        indx = random.randint(0, len(dataset))
        
        #Use gray scale color map if there is only one channel
        #cmap = 'gray' if len(dataset[indx].shape) == 2 else cmap
        # In this case we prob need to use the RGB images.
        #COMPLETE
        plt.imshow(dataset[indx], cmap = cmap)
        plt.xticks([])
        plt.yticks([])
    plt.tight_layout(pad=0, h_pad=0, w_pad=0)
    plt.show()

In [41]:
# plot the sample images
list_images(x_train, y_train)
list_images(x_test, y_test)

NameError: name 'n' is not defined

# Data Pre-processing

In [10]:
# shuffle the order of the training images to randomize
X_train, y_train = shuffle(X_train, y_train)

NameError: name 'X_train' is not defined

In [11]:
# equalize the RGB images
def local_equalize(image):
    # change the size of kernel.
    kernel = morp.disk(30)
    img_local = rank.equalize(image, selem=kernel)
    return img_local

In [12]:
# Sample images after equalization
equalized_images = list(map(local_equalize, COMPLETE))
list_images(equalized_images, y_train, "Equalized Image", "COMPLETE")# Sample images after equalization
equalized_images = list(map(local_equalize, COMPLETE))
list_images(equalized_images, y_train, "Equalized Image", "COMPLETE")

NameError: name 'COMPLETE' is not defined

In [13]:
# normalize the images on a 0 - 1 scale
def image_normalize(image):
    image = np.divide(image, 255)
    return image

In [14]:
# sample images after normalizing
n_training = X_train.shape
normalized_images = np.zeros((n_training[0], n_training[1], n_training[2]))
for i, img in enumerate(equalized_images):
    normalized_images[i] = image_normalize(img)
list_images(normalized_images, y_train, "Normalized Image", "COMPLETE")
normalized_images = normalized_images[..., None]

NameError: name 'X_train' is not defined

# VGG

In [15]:
import torch.nn as nn

In [16]:
class VGG(nn.Module):

    def __init__(self, features, num_classes= COMPLETE, init_weights=True):
        super(VGG, self).__init__()
        self.features = features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)


    def make_layers(cfg, batch_norm=False):
        layers = []
        in_channels = 3
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v
        return nn.Sequential(*layers)
    cfg = {
        'A': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'B': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
        'D': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
        'E': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
    }


    def vgg11(pretrained=False, **kwargs):
        """VGG 11-layer model (configuration "A")
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['A']), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg11']))
        return model


    def vgg11_bn(pretrained=False, **kwargs):
        """VGG 11-layer model (configuration "A") with batch normalization
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['A'], batch_norm=True), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg11_bn']))
        return model


    def vgg13(pretrained=False, **kwargs):
        """VGG 13-layer model (configuration "B")
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['B']), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg13']))
        return model


    def vgg13_bn(pretrained=False, **kwargs):
        """VGG 13-layer model (configuration "B") with batch normalization
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['B'], batch_norm=True), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg13_bn']))
        return model


    def vgg16(pretrained=False, **kwargs):
        """VGG 16-layer model (configuration "D")
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['D']), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg16']))
        return model


    def vgg16_bn(pretrained=False, **kwargs):
        """VGG 16-layer model (configuration "D") with batch normalization
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['D'], batch_norm=True), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg16_bn']))
        return model


    def vgg19(pretrained=False, **kwargs):
        """VGG 19-layer model (configuration "E")
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['E']), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg19']))
        return model


    def vgg19_bn(pretrained=False, **kwargs):
        """VGG 19-layer model (configuration 'E') with batch normalization
        Args:
            pretrained (bool): If True, returns a model pre-trained on ImageNet
        """
        if pretrained:
            kwargs['init_weights'] = False
        model = VGG(make_layers(cfg['E'], batch_norm=True), **kwargs)
        if pretrained:
            model.load_state_dict(model_zoo.load_url(model_urls['vgg19_bn']))
        return model

#  Train the network